In [54]:
import numpy as np
import pandas as pd

In [55]:
def Strategy3(sub_pCTR, sub_basebid):
    avgCTR = 0.5
    sub_pCTR_ = sub_pCTR.copy()
    #sub_pCTR_["pCTR"]=(((sub_pCTR_["pCTR"]+0.5)**2)-0.5)
    #sub_pCTR_["pCTR"] = sub_pCTR_["pCTR"].map(lambda x: 0 if x <0.43 else np.exp(x))
    #sub_pCTR_["pCTR"] = sub_pCTR_["pCTR"].map(lambda x: 0 if x <0.43 else 1)
    #bidprice = (sub_pCTR_["pCTR"] * sub_basebid["predicted_payprice"]) / avgCTR
    bidprice=sub_pCTR_["pCTR"] * 1
    return bidprice

In [56]:
path = 'C:/Users/Akis/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/Test Set Predictions/'
ridge_preds = pd.read_csv(path+"ridge_basebid_test.csv") #, index_col  = "Unnamed: 0"
dwane_preds = pd.read_csv(path+"test_dummy.click_predictions_narrow.2018-03-13.csv")
achilleas_preds = pd.read_csv(path+"xgb_pCTR_test.csv")
#achilleas_preds2 = pd.read_csv(path+"lr_pCTR_test.csv")
#achilleas_preds3 = pd.read_csv(path+"rf_pCTR_test.csv")

In [57]:
#Our best ensemble model by multiplication
combined_preds1 = pd.DataFrame(columns=['bidid','pCTR'])
combined_preds1['pCTR'] = achilleas_preds['pCTR']*dwane_preds['pCTR']
#combined_preds1["pCTR"] = combined_preds1["pCTR"].map(lambda x: 0 if x <0.5 else x)
combined_preds1["pCTR"] = combined_preds1["pCTR"].map(lambda x: 1 if ((0.0   <=x) & (x <0.025)) else 0)
#print(combined_preds1['pCTR'].astype(bool).sum(axis=0))
print(combined_preds1['pCTR'][combined_preds1['pCTR'] > 0].count())



259154


combined_preds2 = pd.DataFrame(columns=['bidid','pCTR'])
#print(achilleas_preds.shape[0]/100)
percentile = 99
#dwane_preds["pCTR"] = dwane_preds["pCTR"].map(lambda x: 0 if x < 1 else np.exp(x))
#print(dwane_preds['pCTR'].astype(bool).sum(axis=0))
#achilleas_preds["pCTR"] = achilleas_preds["pCTR"].map(lambda x: 0 if x <0.44 else x) #0.93
#print(achilleas_preds['pCTR'][achilleas_preds['pCTR'] > 0].count())
limit = np.percentile(achilleas_preds2["pCTR"],percentile)
achilleas_preds2["pCTR"] = achilleas_preds2["pCTR"].map(lambda x: 0 if x < limit else x) #0.9998
print(achilleas_preds2['pCTR'][achilleas_preds2['pCTR'] > 0].count())
limit = np.percentile(achilleas_preds3["pCTR"],percentile)
achilleas_preds3["pCTR"] = achilleas_preds3["pCTR"].map(lambda x: 0 if x <limit else x) #0.87
print(achilleas_preds3['pCTR'][achilleas_preds3['pCTR'] > 0].count())
combined_preds2['pCTR'] = achilleas_preds3['pCTR']*achilleas_preds2['pCTR']#*achilleas_preds['pCTR']
print(combined_preds2['pCTR'][combined_preds2['pCTR'] > 0].count())

combined_preds = pd.DataFrame(columns=['bidid','pCTR'])
print(combined_preds1['pCTR'][combined_preds1['pCTR'] > 0].count())
print(combined_preds2['pCTR'][combined_preds2['pCTR'] > 0].count())
combined_preds['pCTR'] = combined_preds1['pCTR']+combined_preds2['pCTR']
print(combined_preds['pCTR'][combined_preds['pCTR'] > 0].count())
print("Additional Bids:",combined_preds['pCTR'].astype(bool).sum(axis=0)-combined_preds1['pCTR'].astype(bool).sum(axis=0))

In [58]:
#combined_preds = pd.DataFrame(columns=['bidid','pCTR'])
#combined_preds['pCTR'] = achilleas_preds['pCTR']*dwane_preds['pCTR']
#combined_preds['bidid'] = dwane_preds['bidid']

In [59]:
combined_preds = combined_preds1.copy(True)

In [60]:
bids = pd.DataFrame(columns=['bidid','bidprice'])
#bids['bidprice'] = Strategy3(combined_preds,ridge_preds)
bids['bidprice'] = combined_preds['pCTR'] * 100000 #*2621 *130 *10
print(combined_preds['pCTR'][combined_preds['pCTR'] > 0].count())
bids['bidid']=ridge_preds['bidid']
#bids['bidprice'] = bids['bidprice'] * 2621 *5 *10 *10000
#bids['bidprice'] = bids['bidprice'].round(0)
#bids['bidprice'] = bids['bidprice'].astype(int)
print(bids['bidprice'][bids['bidprice'] > 0].count())
#bids.tail(1)['bidprice']=6250000-6247377
#bids['bidprice'][-1]=6250000-6247377
print()

259154
259154



In [50]:
bids['bidprice'].sum()

1224800000

In [51]:
# pd.options.display.max_rows=290
# bids.tail(100)
# print(bids['bidprice'].astype(bool).sum(axis=0))
# print(bids['bidprice'][bids['bidprice'] > 0].count())

In [52]:
#print(bids.shape)
#ourAverage = bids['bidprice'].mean()
#print("Previous:",154240.75)
totalBid = bids['bidprice'].sum()
print("totalBid",totalBid)
totalBidsPlaced = bids['bidprice'][bids['bidprice'] > 0].count()
ourAverage = bids['bidprice'].sum()/totalBidsPlaced
print("ourAverage",ourAverage)
print("totalBid",totalBid)
print("totalBidsPlaced",totalBidsPlaced)
print("Multiples of budget bid:",totalBid/6250000)

totalBid 1224800000
ourAverage 100000.0
totalBid 1224800000
totalBidsPlaced 12248
Multiples of budget bid: 195.968


In [53]:
bids.to_csv('Group_21_SingleAgentBids.csv',index=False)

In [ ]:
averageNeeded = 659 #659 is sum(budgets of all teams)/number of impressions available; assumes 32 teams (there should be 31 but 32 plays it safe)

In [ ]:
bids['bidprice'] = bids['bidprice']*(averageNeeded/ourAverage)
bids['bidprice'] = bids['bidprice'].round(0)
bids['bidprice'] = bids['bidprice'].astype(int)
print(bids.head(5))

In [ ]:
ourAverage = bids['bidprice'].mean()
totalBid = bids['bidprice'].sum()
print("ourAverage",ourAverage)
print("totalBid",totalBid)
print("Multiples of budget bid:",totalBid/6250000)

In [ ]:
bids.to_csv('Group_21_MultiAgentBids.csv',index=False)